# Property appraisal ML project.
## Phase 1: developing some similarity-measure-based ranking algorithm

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance

In [2]:
# reading one file
# df = pd.read_csv('F20 P1.csv', index_col=None, header=0)

# reading part of the files from Detached June 2020-2021 folder

data_files = ['F20 P1.csv', 'F20 P2.csv', 'F30 P1.csv', 'F30 P2.csv', 'F50 P1.csv', 'F50 P2.csv']
data_list = []

for filename in data_files:
    df_current = pd.read_csv(filename, index_col=None, header=0)
    data_list.append(df_current)

df = pd.concat(data_list, axis=0, ignore_index=True)

# deleting yellow columns
df.drop(['Status', 'For Tax Year', 'Gross Taxes', 'Original Price', 'List Price', 'GST Incl'], axis = 1, inplace = True)

# size of our dataset
print('Our dataset has', len(df), 'data lines and', len(df.columns.tolist()), 'features:')
print('\n')
print(df.columns.tolist())

Our dataset has 5844 data lines and 23 features:


['Address', 'S/A', 'Price', 'Sold Date', 'Days On Market', 'Age', 'Area', 'Total Bedrooms', 'Total Baths', 'Lot Sz (Sq.Ft.)', 'Floor Area -Grand Total', 'Driveway Finish', 'Floor Area - Unfinished', 'Foundation', 'Floor Area Fin - Basement', 'Zoning', 'Parking Places - Covered', '# Rms', 'No. Floor Levels', 'Frontage - Feet', 'Depth', 'Type', 'Public Remarks']


In [3]:
# dropping the columns with more than 90% NAs
moreThan = []

for feature in df:
    if df[feature].isna().sum() / df.shape[0] > 0.9:
        moreThan.append(feature)
        print("Dropping the feature:", feature)
df.drop(moreThan, axis = 1, inplace = True)

if moreThan == []:
    print('No features dropped.')
print('\n')

# dropping the columns that are not insightful: Days On Market, Public Remarks
# df1.drop(['Sold Date', 'Public Remarks'], axis=1, inplace = True)
# df.drop(['Public Remarks'], axis=1, inplace = True)

columns_names = df.columns.tolist()

print("Features left:")
print(columns_names)
print('\n')
print("Now we have", len(columns_names), "features and their types:")

# types of our columns
pd.DataFrame(df.dtypes, columns=['DataTypes'])

No features dropped.


Features left:
['Address', 'S/A', 'Price', 'Sold Date', 'Days On Market', 'Age', 'Area', 'Total Bedrooms', 'Total Baths', 'Lot Sz (Sq.Ft.)', 'Floor Area -Grand Total', 'Driveway Finish', 'Floor Area - Unfinished', 'Foundation', 'Floor Area Fin - Basement', 'Zoning', 'Parking Places - Covered', '# Rms', 'No. Floor Levels', 'Frontage - Feet', 'Depth', 'Type', 'Public Remarks']


Now we have 23 features and their types:


,DataTypes
Address,object
S/A,object
Price,object
Sold Date,object
Days On Market,int64
Age,float64
Area,object
Total Bedrooms,int64
Total Baths,int64
Lot Sz (Sq.Ft.),object


In [4]:
df1 = df.copy()

df1.drop(['Public Remarks'], axis=1, inplace = True)
df1.drop(['Address'], axis=1, inplace = True)

df1['Price'] = df1['Price'].str.replace('$','').str.replace(',','').astype(int)

df1['Sold Date'] = pd.to_datetime(df1['Sold Date'])

df1['Age'] = df1['Age'].fillna(-1).astype(int)
df1.loc[df1['Age'] > 150, 'Age'] = 150

df1['Lot Sz (Sq.Ft.)'] = df1['Lot Sz (Sq.Ft.)'].str.replace(',', '').astype(float)
# df1.drop(df1[df1['Lot Sz (Sq.Ft.)'] > 50000].index, inplace = True)
# df1.drop(df1[df1['Lot Sz (Sq.Ft.)'] < 100].index, inplace = True)

df1['Floor Area -Grand Total'] = df1['Floor Area -Grand Total'].str.replace(',', '').astype(int)

df1['Driveway Finish'] = df1['Driveway Finish'].astype(str)

df1['Floor Area - Unfinished'] = df1['Floor Area - Unfinished'].str.replace(',', '').astype(int)

df1['Foundation'] = df1['Foundation'].astype(str)

df1['Floor Area Fin - Basement'] = df1['Floor Area Fin - Basement'].str.replace(',', '').astype(float)

df1['Zoning'] = df1['Zoning'].astype(str)
df1['Zoning'] = df1['Zoning'].str.replace('A1', 'A-1')
df1['Zoning'] = df1['Zoning'].str.replace('A2', 'A-2')
df1['Zoning'] = df1['Zoning'].str.replace('1ACRER', 'RA')
df1['Zoning'] = df1['Zoning'].str.replace('1 AR', 'RA')
df1['Zoning'] = df1['Zoning'].str.replace('RF13', 'RF-13')
df1['Zoning'] = df1['Zoning'].str.replace('RHG', 'RH-G')
df1['Zoning'] = df1['Zoning'].str.replace('RS-1', 'RS1')
df1['Zoning'] = df1['Zoning'].str.replace('SING/F', 'SING')

df1['Parking Places - Covered'] = df1['Parking Places - Covered'].fillna(-1) ### or -1

df1.loc[df1['No. Floor Levels'] > 10, 'No. Floor Levels'] = -1

df1['Frontage - Feet'] = df1['Frontage - Feet'].str.replace(',', '').astype(float)
df1['Frontage - Feet'] = df1['Frontage - Feet'].fillna(-1) ### or -1

df1 = df1.drop(['Depth'], axis=1)

df1['new feature 1'] = (df1['Total Bedrooms'] + df1['# Rms'])

df1['Total Baths'] = df1['Total Baths'].astype(str)
df1['# Rms'] = df1['# Rms'].astype(str)

In [5]:
# this function preprocesses the dataset by doing the following:
# 1. drops "S/A" and "Area" columns as they are not needed (all the houses from the same neighbourhood)
# 2. making similar one-hot-encoding as in the baseline sloution and polynomial reduction of numerical columns
# 3. MinMax scaling the data: this is important as we're going to utilize a distance metrics which is sensible to scales of different features

def preproc_df(dataframe):
    dataframe1 = dataframe.copy()
    dataframe1 = dataframe1.drop(['S/A', 'Area'], axis=1)
    X1, X2, X, X_sold_date, y = [], [], [], [], []
    
    X_sold_date = dataframe1['Sold Date']
    y = dataframe1['Price']

    # one-hot-encoding categorical features
    X1 = pd.get_dummies(dataframe1[[ 'Total Baths', '# Rms', 'Driveway Finish', 'Foundation', 'Type', 'Zoning']])

    X2 = dataframe1.drop(['Price', 'Sold Date', 'Total Baths', '# Rms', 'Driveway Finish', 'Foundation', 'Type', 'Zoning'], axis = 1)
    X2.fillna(0)
    
#     poly = PolynomialFeatures(2)
#     X2_2 = poly.fit_transform(X2)

#     X2_3 = pd.DataFrame(X2_2, columns = poly.get_feature_names(X2.columns))
#     X2_3 = X2_3.drop(['1'], axis = 1)
    
    # generating a big preprocessed dataset
    X = pd.concat([X1, X2], axis = 1)

    #rescaling it
    X[X.columns] = MinMaxScaler().fit_transform(X[X.columns])
    
    X = pd.concat([X, dataframe1['Sold Date'], dataframe1['Price']], axis = 1)
    
    return X

In [6]:
# this function generates the list of recent sales from the same neighbourhood for the given property:
# 1. first it gets the list of recent (90 days prior to given property sale date) sales properties
# 2. if this list has less or equal than 3 members - then it generates a list of sales from -90 to +90 days of sales

def get_last_sales_df(prop, df_same_subarea_preprocessed):
    df_same_subarea_2, last_90_days_sold_list, final_df = [], [], []
    
#     prop = df_same_subarea_preprocessed.loc[idx]
    df_same_subarea_2 = df_same_subarea_preprocessed.copy()
#     df_same_subarea_preprocessed_dropped = df_same_subarea_preprocessed_dropped.drop(idx)
    
    last_90_days_sold_list = df_same_subarea_2[((prop['Sold Date'] - df_same_subarea_2['Sold Date']) <= pd.Timedelta('90 days 00:00:00')) 
                                                     & ((prop['Sold Date'] - df_same_subarea_2['Sold Date']) >= pd.Timedelta('0 days 00:00:00'))] 
    
    if len(last_90_days_sold_list) >= 3:
        final_df = last_90_days_sold_list
    else:
        final_df = df_same_subarea_2[((prop['Sold Date'] - df_same_subarea_2['Sold Date']) <= pd.Timedelta('90 days 00:00:00')) 
                                                     & ((prop['Sold Date'] - df_same_subarea_2['Sold Date']) >= pd.Timedelta('-90 days 00:00:00'))]
#     else nothing???

    return final_df.drop(['Sold Date'], axis=1)

In [7]:
# this function generates distance between the vectors of given property and other properties from the same neighbourhood and correct sales days in n-dimensional plane
# it uses L1 norm as after playing a lot with different distances (cosine, L2, others) I found out a correlation between L1 norm and similarity of prices.
# for L2 or cosine distances the correlation is not so obvious. 
# also, I found out a justification of L1 norm usage in one article on similar topic.

def dist_matrix_cal(prop, last_sales_df):
    dist_matrix = []
    indices_list = []
    indices_list = last_sales_df.index

    for i in range(len(last_sales_df)):
#         dist_matrix.append(distance.cosine(prop, last_sales_df.iloc[i]))
        dist_matrix.append(np.linalg.norm(prop - last_sales_df.iloc[i], ord=1))
    
    return indices_list, dist_matrix

In [8]:
# this function is main building block:
# for a given property (index!) it generates the last sales list then generates the list of k most similar properties with similarity score

def get_similarity_score(prop_index, dataframe, k_similar_properties):
    dataframe1 = []
    prop, prop_SA, prop_price, dist_prop, prop_indexes = [], [], 0, [], []
    df_same_subarea, prop_last_sales_df, prop_last_sales_prices = [], [], []
    
    # copy the initial dataframe
    dataframe1 = dataframe.copy()
    
    # get the given property S/A
    prop_SA = dataframe1.loc[prop_index]['S/A']
    
    # generate the dataframe of properties from the same neighbourhood 
    df_same_subarea = dataframe1[dataframe1['S/A'] == prop_SA]
    df_same_subarea = preproc_df(df_same_subarea)
    
    # get all the needed info for the given property
    prop = df_same_subarea.loc[prop_index]
    prop_price = prop['Price']
    prop = prop[:-1]
    
    # and drop it from the general dataset
    df_same_subarea = df_same_subarea.drop(prop_index)
    
    # generate the list of recent sales similar to the given prop
    prop_last_sales_df = get_last_sales_df(prop, df_same_subarea)
    prop_last_sales_prices = np.array(prop_last_sales_df['Price'])
    prop_last_sales_df= prop_last_sales_df.drop(['Price'], axis=1)
    prop = prop[:-1]

    # get the k_similar_properties most similar properties and distances to them 
    prop_indexes, dist_prop = dist_matrix_cal(prop, prop_last_sales_df)
    numbers = np.argsort(dist_prop)[:k_similar_properties]
    distances = np.round((np.sort(dist_prop))[:k_similar_properties], 2)

    print('Property index:', prop_index)
    print('The price of this property:', prop_price)
    print('Indexes of the', k_similar_properties, 'most similar properties:', np.array(prop_indexes[numbers]))
    print('Distances to the', k_similar_properties, 'most similar properties:', distances)
    print('Prices of the', k_similar_properties, 'most similar properties:   ', prop_last_sales_prices[numbers], '\n')

In [9]:
# here I demonstrate how these functions work:
# I pick a random property in a random area
# and generate the list of 3 (6) most similar properties and similarity score

idx = 221

# 3 most similar properties
get_similarity_score(idx, df1, 3)

# 6most similar properties
get_similarity_score(idx, df1, 6)

Property index: 221
The price of this property: 975000
Indexes of the 3 most similar properties: [  6 544 165]
Distances to the 3 most similar properties: [10.43 10.78 10.93]
Prices of the 3 most similar properties:    [ 716000 1882000  935000] 

Property index: 221
The price of this property: 975000
Indexes of the 6 most similar properties: [  6 544 165 525 556]
Distances to the 6 most similar properties: [10.43 10.78 10.93 12.48 14.89]
Prices of the 6 most similar properties:    [ 716000 1882000  935000 1725000 2550000] 



In [10]:
# now I demonstrate the same for say 10 randomly picked different properties:

idx = np.random.randint(len(df1), size=10)

for i in idx:
    get_similarity_score(i, df1, 6)

Property index: 2990
The price of this property: 1275000
Indexes of the 6 most similar properties: [3047 2871 2729 2893 3354 2649]
Distances to the 6 most similar properties: [3.   4.61 4.63 4.65 4.66 4.74]
Prices of the 6 most similar properties:    [1300000 1225000 1115000 1231000 1450000 1050000] 

Property index: 589
The price of this property: 905000
Indexes of the 6 most similar properties: [568 629  44  34 672 647]
Distances to the 6 most similar properties: [2.67 3.11 3.2  3.41 5.08 5.17]
Prices of the 6 most similar properties:    [ 762800  995000  822000  805000 1060000 1020000] 

Property index: 2175
The price of this property: 1304761
Indexes of the 6 most similar properties: [2141 2119 2210 2162 2158 2059]
Distances to the 6 most similar properties: [2.06 4.33 4.71 4.83 6.24 6.33]
Prices of the 6 most similar properties:    [1285714 1275000 1325000 1299000 1296000 1240000] 

Property index: 2421
The price of this property: 1640000
Indexes of the 6 most similar properties: 